In [ ]:

import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import os
from PIL import Image
from io import BytesIO
import time
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard


In [ ]:
from tensorflow.python.client import device_lib

In [ ]:
def video_reader(cv2, filename):
    frames = np.zeros((30, 160, 160, 3))
    i = 0
    print(frames.shape)

    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval, frame = vc.read()
    else:
        rval = False

    frm = resize(frame, (160, 160, 3))
    frm = np.expand_dims(frm, axis=0)
    if np.max(frm) > 1:
        frm = frm / 255.0

    frames[i][:] = frm
    i += 1
    print("reading video")

    while i < 30:
        rval, frame = vc.read()
        frm = resize(frame, (160, 160, 3))
        frm = np.expand_dims(frm, axis=0)
        if np.max(frm) > 1:
            frm = frm / 255.0
        frames[i][:] = frm
        i += 1

    return frames


In [ ]:
def surveillance_model2(tf, weight='/content/drive/MyDrive/best_weights.hdfs'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics

    num_classes = 2

    input_shape = (160, 160, 3)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False, weights='imagenet', input_shape=input_shape)

    cnn = models.Sequential()

    cnn.add(base_model)
    cnn.add(layers.Flatten())

    model = models.Sequential()

    model.add(layers.TimeDistributed(cnn, input_shape=(30, 160, 160, 3)))

    model.add(layers.LSTM(30, return_sequences=True))

    model.add(layers.TimeDistributed(layers.Dense(90)))
    model.add(layers.Dropout(0.1))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation="sigmoid"))

    adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(weight)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model

In [ ]:
import numpy as np
from skimage.transform import resize
np.random.seed(1234)
model22= surveillance_model2(tf)

In [ ]:

def pred_violence(model, video, accuracy=0.9):
    pred_test = model.predict(video)
    if pred_test[0][1] >= accuracy:

        return True, pred_test[0][1]
    else:
        return False, pred_test[0][1]


In [ ]:
def main_surveillance(video):
    vid = video_reader(cv2,video)
    datav = np.zeros((1, 30, 160, 160, 3))
    datav[0][:][:] = vid
    millis = int(round(time.time() * 1000))
    print(millis)
    f , precent = pred_violence(model22,datav,accuracy=0.65)
    millis2 = int(round(time.time() * 1000))
    print(millis2)
    res = {'violence':f , 'precentegeofviolence':str(precent)}
    res['processing_time'] =  str(millis2-millis)
    return res

In [ ]:
res = main_surveillance('/content/drive/MyDrive/v.mp4')
print(res)

(30, 160, 160, 3)
reading video
1714576216537
1/1 [==============================] - 13s 13s/step
1714576230038
{'violence': False, 'precentegeofviolence': '0.004806572', 'processing_time': '13501'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tensorboard --logdir logs/
